In [0]:
!pip install pyspark

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('practice').getOrCreate()

data = [(1,'krish') , (2,'ajay'),(3,'james')]
columns = [ 'id' , 'name' ]

df = spark.createDataFrame(data,columns)
df.show()
df.printSchema()


In [0]:
#header = True: treat the first row of the csv as headers, if header = False then the first row will _c01, _c02
#inferschema = True: understand the schema of hte table from csv if its false then treat all the data string.

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('practice').getOrCreate()

df = spark.read.csv('/content/sample_data/california_housing_test.csv',header=True,inferSchema=True)

df.show()
#df.show()
df.printSchema()
#_c01 _c02

In [0]:
newdf = df.filter(df['longitude']<-123)
newdf.show()


In [0]:
#withColumn: add a new column
df= df.withColumn('sumdata',df['longitude']+df['latitude'])
df.show()

In [0]:
#groupby : applied when we aggregating data



In [0]:
ordersdf = spark.read.csv('/content/orders.csv', header  = True, inferSchema=True)

In [0]:
ordersdf.show()
#type(ordersdf)

In [0]:
customer_df = spark.read.csv('/content/customers.csv',header= True, inferSchema=True)

In [0]:
#join dataframes
mergeddf = ordersdf.join(customer_df,on='customerid', how = 'inner')
newdf= mergeddf.select('CustomerID','Name','Quantity','Country')
newdf.show()

In [0]:
#sortdata
newdf = mergeddf.orderBy('Quantity', ascending=True)
newdf.show()

In [0]:
#drop duplicates
newdf = mergeddf.dropDuplicates(['customerid'])
newdf.show()


In [0]:
#udf: user defined function
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType,IntegerType, FloatType
def to_uppercase(s):
  return s.upper()


In [0]:
myudf = udf(to_uppercase, StringType())

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, FloatType

def to_uppercase(s):
  return s.upper()

myudf = udf(to_uppercase, StringType())

mergeddf = ordersdf.join(customer_df, on='customerid', how='inner')


#download udf
#create a udf object with output data type and function u need to apply
#create a new column and pass the udf with the parameter as the columns u to apply it to.




In [0]:
mergeddf.show()
mergeddf.withColumn('uppercase_name', myudf('Name')).show()

In [0]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

def totalprice(quantity, price):
  return quantity * price

# Use FloatType if price or quantity might be floats
myudf = udf(totalprice, FloatType())


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def totalprice(quantity, price):
    return quantity * price

myudf = udf( totalprice , IntegerType())

udf
mergeddf = mergeddf.withColumn('total_price', myudf( mergeddf['quantity'], mergeddf['price'] ))

mergeddf.show()

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('practice').getOrCreate()

data = [
    (1, 'Alice', 'HR', 50000),
    (2, 'Bob', 'HR', 60000),
    (3, 'Charlie', 'Sales', 55000),
    (4, 'David', 'Marketing', 45000),
    (5, 'Eve', 'Finance', 70000),
]

columns = ['employee_id', 'employee_name', 'department', 'salary']

employee_df = spark.createDataFrame(data, columns)

employee_df.show()


#with cte as (select employeename, salary, department, dense_rank() over (parition by department order by salary desc) as ranking
#from employees)

#select * from cte where ranking =2;

In [0]:
# cte
# partition by department
# order by salary desc
# dense_rank() over (partition by department order by salary desc) as ranking


select * from cte
where ranking = 2;


In [0]:

from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, lead,lag,rank,row_number, col

window_spec  = Window.partitionBy(col('department')).orderBy(col('salary').desc())
ranked_df = employee_df.withColumn('rank', dense_rank().over(window_spec))
ranked_df.show()


#window : pyspark.sql.window | pyspark.sql.functions | pyspark.sql.types import IntegerType, StringType, FloatType |


In [0]:
secondhighestsalary = ranked_df.filter( col('rank')==2 ).show()

In [0]:
#second highest salary by department
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('practice').getOrCreate()

from pyspark.sql import Window
from pyspark.sql.functions import col,dense_rank

data = [
    (1, 'Alice', 'HR', 50000),
    (2, 'Bob', 'Engineering', 60000),
    (3, 'Charlie', 'HR', 55000),
    (4, 'David', 'HR', 45000),
    (5, 'Eve', 'Engineering', 70000),
]

columns = ['employee_id', 'employee_name', 'department', 'salary']

employee_df = spark.createDataFrame(data, columns)

windowspec = Window.partitionBy('department').orderBy(col('salary').desc())

ranked_df = employee_df.withColumn('rank', dense_rank().over(windowspec))

second_highest_df = ranked_df.filter( col('rank') == 2 )

second_highest_df.show()



In [0]:
employee_df.createOrReplaceTempView('employee')

spark.sql('select * from employee where department= "HR"').show()
spark.sql('select * from employee where department= "IT"').show()

In [0]:
spark.sql("select department, avg(salary) as avgsalary from employee group by department").show()

In [0]:
spark.sql("select * from employee order by salary desc").show()

In [0]:
query = """ select department, employee_name, salary from (select department, employee_name, salary, dense_rank()
over (partition by department order by salary desc) as rank from employee) where rank = 2"""

spark.sql(query).show()

In [0]:
#most frequently asked interview question : extract data from one source to another apply simple transformation
# Data source format
# Transformattion to be applied
# use case : why are we doing this ?
# final format of data?
# frequency of data
# size

#SQL server : connections

# define the source link: link = 'mylink/data.csv'
# destinationpath = 'mypath/newdata.parquet'
# extract: spark.read.csv(link)
# df.drop_duplicates()
# df.withColumn('city',lit('bangalore'))
# df.write.parquet(destinationpath)


from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
spark = SparkSession.builder.appName('practice').getOrCreate()


df = spark.read.csv('mylink/data.csv')
df.dropDuplicates()
dfwithnewco = df.withColumn('city', lit('bangalore'))
dfwithnewco.show()

# id name age city
# 1 a 16 bangalore
# 2 b 16 bangalore
# 3 c 18 bangalore



#scenario 2 : data is sitting in sql server

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('practice').getOrCreate()

import mysqlconnector-python
mysqlurl = 'myurl'

# employeetable = 'employee'
# departmenttable = 'department'
mysqltable = ['employee','department']

mysqlproperties =
{
    'host':'myhost',
    'user':'root',
    'password':'password',
    'driver': 'mydriver'
}


#interview version

mysqlurl = 'myurl'
mysqltables= table1

employeedf =   spark.read.jdbc(url=mysqlurl, table=mysqltable[0], properties=mysqlproperties)
departmentdf = spark.read.jdbc(url=mysqlurl, table=mysqltable[1], properties=mysqlproperties)

employeedf.show()
departmentdf.show()

#joined the dataframes
mergeddf = employeedf.join(departmentdf, employeedf.departmentid == departmentdf.departmentid, how = 'inner')

#change the name of the column : withColumnRenamed(col('name),'newname)

#drop duplicates
mergeddf.dropDuplicates(['employeeid']).show()
df.withColumn('city', lit('bangalore'))
mergeddf.write.parquet('path')




In [0]:
#we have data in a database , extract data (employee and department).

sqlproperties= {
url:myurl
id:admin
password:password
host:localhost
driver:com.mysql.cj.jdbc.Driver
}

tables = ['employee','department']

employee_df = spark.read.jdbc(url = url, table = 'employee', properties = sqlproperties)
department_df = spark.read.jdbc(url = url, table = 'department', properties = sqlproperties)

#join hte data and tranform

newdf  = employeedf.join(departmetndf, employeedf.departmentid == departmentdf.departmentid, how = 'inner')

newdf.withColumn('city', lit('bangalore'))

#write the data in parquet format in a location

newdf.write.parquet(path)


In [0]:
#read data from csv
#apply udf
#filter
#join
# window functions
#create a new column
#read data from json or csv
#read data from databse  : jdbc connection  spark.read.jdbc
#write data into parquet


In [0]:
!pip install mysql-connector-pythonc

import mysql.connector #mysql database
import psycopg2 #redshift | postgressql
import sqlalchemy #anything generic


config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'password',
    'database': 'database'
}

connection = myurl
cursor = connection.cursor()

query = 'select * from employee'
cursor.execute(query)
result = cursor.fetchall()

for row in result:
  print(row)


In [0]:
#internal SQL databases
#1) email ingestion
#2) dump the csv file into a folder any folder (SFTP -secure file transfer protocol)
#3) API integration (advantage it can customised and secured, disadvantage: money to develop and manage)
#4) JDBC - plug and play.


In [0]:
from sqlalchemy import create_engine
import pandas as pd

mysqlurl = 'jdbc:mysql://localhost:3306/database'
mysqltable = 'employee'

engine = create_engine(mysqlurl)

df = pd.read_sql_table(mysqltable, engine)

df.head()



In [0]:
#psyopg2 - postgres sql.

import psycopg2

connection = {

    host = 'localhost',
    database =  'mydatabase'
    user = 'myuser',
    password = 'mypassword'
    port='5432'
}

connection = psycopg2.connect(**connection)
cursor = connection.cursor()

query = 'select * from employee'
cursor.execute(query)
result = cursor.fetchall()

cursor.close()
connection.close()

for row in result:
  print(row)



In [0]:
#exact Interview coding questions:

#Initialise and import relevant libraries
#how to import from database, s3bucket, datawarehouse (mysql, postgressql, s3bucket, jdbc connections)
#tranformations: create a new column, filter data,select data,sort, drop duplicates, merge dataframes, apply window function for second highest salary, UDF
#write data in csv format, parquet format or jdbc connection to a database (output)



In [0]:
##example real world with coalesce, repartition, cache, persist, broadcast joins.

#ecommerce:
#usecase: analyse customer behaviour to provide custom offers and deals depending on customer behaviour|
#daily transactional dashboard (sales, returns, profits, losses, top selling item, lowest selling items
#top selling location, least selling location, Top selling category, least selling category, male vs female shoppers )


#Data sources

#internal database:JDBC: mysql database (transaction, customer, product, logistics)
#external data: API: api integration into google (trends , analytics), facebook (marketing spend, competitor marketing data) and 3rd party api's (benchmarking -competitor pricing ($1))
#Internal / external : SFTP(secure file transfer protocol) Folder :  - csv data: from individual departments ( customer survey, marketing department) :pysftp

#orders | customer | product|google analytics(external)

#3 tables:

#1) total orders by customer (orders, customer)
#2) amount spent by customer (orders and customers)
#3) date and time of the order by customer (orders)
#4) product category(product, orders, customer)
#5) location of customer (customer)
#6) popular product based on location, brand and pricing.(product, orders, customer)
#7) spend by gender,age, location, category.(orders, customer,product)

#Data engineer goals:
# I wrote scripts to extract the data from jdbc and sftp
# once the data was extracted i was responsible for the below:

# join the data set to get a consolidated view of purchased
# apply tranformations to clean and filter the data
# optimise the pipeline for large scale processing using coalesce, repartition, cache, persist and broadcast joins.


#customer: customerid | name | city

#orders : orderid | productid | quantity| orderdate|customerid

#product: productid | name| category | price


from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit

spark = SparkSession.builder.appName('practice').getOrCreate()

customers_data = [(1, 'Alice', 'New York'), (2, 'Bob', 'San Diego'), (3, 'Charlie', 'Seattle')]
orders_data = [(101, 1, 201, 2, '2023-12-25'), (102, 2, 202, 1, '2023-12-26'), (103, 3, 203, 3, '2023-12-27')]
products_data = [(201, 'Laptop', 'Electronics', 1000), (202, 'Smartphone', 'Electronics', 700), (203, 'Headphones', 'Accessories', 150)]


s3bucket = 'path/rawfolder/data.csv'
accesskey = 'accesskey'
secretkey = 'secretkey'

#step 1: automatically extract data into your df

customerdf = spark.read.csv('s3bucket',header = True, inferSchema=True)
ordersdf = spark.read.csv('s3bucket',header = True, inferSchema=True)
productsdf = spark.read.csv('s3bucket',header = True, inferSchema=True)

#step2: connect using jdbc

url = 'jdbc:mysql://localhost:3306/database'
connectionproperties = {
    'user':username,
    'password':password,
    'host' = 'host'
    'driver':'com.mysql.cj.jdbc.Driver'
}

#option 1: read data directly from the database using jdbc
customerdf = spark.read.jdbc(url='url', user= connectionproperties['username'], password=connectionproperties['password'], configuration= connectionproperties)
ordersdf = spark.read.jdbc(url='url', user= connectionproperties['username'], password=connectionproperties['password'], configuration= connectionproperties)
productsdf = spark.read.jdbc(url='url', user= connectionproperties['username'], password=connectionproperties['password']sword, configuration= connectionproperties)


# option2 :manual dataframe creation
customers_df = spark.createDataFrame(customers_data, ['customer_id', 'customer_name', 'city'])
orders_df = spark.createDataFrame(orders_data, ['order_id', 'customer_id', 'product_id', 'quantity', 'order_date'])
products_df = spark.createDataFrame(products_data, ['product_id', 'product_name', 'category', 'price'])

customers_df.show()
orders_df.show()
products_df.show()


#broadcast join - broadcast smallest dataframe to each node to reduce shuffling.
products_broadcast = products_df.broadcast()

#join the data

order_details_df= orders_df.join(products_broadcast, on='product_id', how='inner')

filtered_df = order_details_df.filter(col('quantity')>1)
newdf = filtered_df.withColumn('total_price', col('quantity') * col('price'))

#cache results for reuse
newdf.cache()

#aggregration

aggregated_df = newdf.groupBy('city', 'category').sum('total_price')

#repartition data for further processing

repartition_df  = aggregated_df.repartition(4)

#run the remaining aggregrations here

#coalesce to optimise partitions for writing

finaldf= repartition_df.coalesce(1)

#write_output

finaldf.write.mode('overwrite').parquet('/content/output')

#1) total orders by customer
#2) amount spent by customer
#3) date and time of the order by customer
#4) product category
#5) location of customer
#6) popular product based on location, brand and pricing.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

spark = SparkSession.builder.appName('practice').getOrCreate()

df = spark.read.parquet('file.parquet')

dfwithnewco = df.withColumn('city', lit('bangalore'))

dfwithnewco.show()

In [0]:
#JDBC integration with mysql > s3 > redshift
#API integration to 3rd party api
#sftp integration
#SQL: removing duplicates (row number, having, distinct)
#pandas df and pyspark df
#RDD vs df
#joins in pyspark
#broadcast variable / join

#working for a ecommerce firm, we are getting data from multiple sources which includes sftp folder. the csv files from sftp are moved to s3 bucket, from s3 bucket extract the data into a sparf df
#write tranformations  : remove duplicates, count of na in each column, fillna, create a new column to run some aggregrations.
#i want to write the file to 2 places. one is redshift table and another is an s3 bucket. i want the file to be writter in parquet format.
# I want to create a dataframe manually

#initialise spark

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, when, lit, count, sum, avg, max, min, countDistinct

spark = SparkSession.builder.appName('practice').getOrCreate()


schema = StructType([
    StructField('customer_id', IntegerType(), True),
    StructField('customer_name', StringType(), True),
    StructField('city', StringType(), True),
    StructField('age', IntegerType(), True),
    StructField('gender', StringType(), True),
    StructField('salary', IntegerType(), True)
])

data = [
    (1, 'Alice', 'New York', 25, 'Female', 50000),
    (2, 'Bob', 'San Diego', 30, 'Male', 60000),
    (3, 'Charlie', 'Seattle', 35, 'Male', 55000),
    (4, 'David', 'Los Angeles', 28, 'Male', 45000),
    (5, 'Eve', 'Chicago', 32, 'Female', 70000),
    (6, 'Frank', 'Miami', 29, 'Male', 55000),
    (7, 'Grace', 'San Francisco', 31, 'Female', 65000),
    (8, 'Henry', 'Boston', 27, 'Male', 52000)
       ]

df = spark.createDataFrame(data, schema)

df.printSchema()

#s3 credentials

s3bucket = 'path/rawfolder/data.csv'
accesskey = 'accesskey'
secretkey = 'secretkey'

#redshift credentials:

redshift_table = 'tablename'
redshift_cluster = 'clustername'
redshift_url = 'jdbc:redshift://endpoint:5439/dbname'
username= 'redshiftusername'
password = 'password'


#count of nulls

namecount = df['name'].isnull().sum()

for i in df.columns:
  print(i, df[i].isNull().sum())


#drop duplicates
df = df.dropDuplicates()

#fillna

df = df.fillna({'age':30, 'salary':50000})

#create a new column salary category

df = df.withColumn(
    'salarycategory',
    when(col('salary') < 50000, 'Low').when((col('salary') >= 50000) & (col('salary') < 60000), 'Medium').otherwise('High')
)

df.show()


s3_target_path = 's3://bucket/data/'
df.write.mode('overwrite').parquet(s3_target_path)

#write to redshift

df.write.format('jdbc').option('url',redshift_url).option('dbtable', redshift_table).option('user', username).option('password', password).mode('overwrite').save()

print(' the data was succesffull saved ')





In [0]:
#read a file from a s3 bucket, Its a csv file and run some minor tranformation on it and move it into anothe s3 bucket. use boto3 library.



In [0]:
#read a file from a s3 bucket which has date in the name of the file, we only need to read the file that has today's date
#and any other dates we need to move the file to archive folder.#after reading the file we need to run some minor tranformation and move the file to another s3 bucket.

#initiate spark session

#naming convention
rawdata\
  logistics\
  customersurvey\
  finance\
    08092025\
    09092025\
      finance_09092025


from pyspark.sql import SparkSession
import boto3
from datetime import datetime

spark = SparkSession.builder.appName('test').getOrCreate()

#mention the location of the file which is s3path
sourcebucket = 'mybucket/source'
destinationbucket = 'mybucket/destination'
archivebucket = 'mybucket/archive'


#calculte today'date
today = datetime.now().strftime('%d%m%Y')

#initiate boto3

s3client = boto3.client('s3')

#get all the files in my s3 bucket


response = s3client.list_objects_v2(bucket= sourcebucket)

filelist = {'data_01012025':1,'data_02012025':1}

#check if there are duplicate files in my s3 bucket if its there then delete the second file.
for i in response['Contents']:
  key = i['Key']
  if key in filelist:
    s3client.delete_object(Bucket=sourcebucket, Key=key)
    print(f'File {key} moved to archive bucket.'


#read the file after duplicate files are deleted and then if the today date is in the key name then read and tranform and
#load to destination or copy to archive and delete at source.

for obj in response['Contents']:
  key =obj['Key']

  #look for today's date in the filename

  if today in key:

    #if today's date in file name read the file.

    df=spark.read.csv('path', header=True, inferSchema= True)

    #drop duplicates

    newdf =df.withColumn('city', lit('bangalore'))


    #export the df to anotehr bucket.
    newdf.write.parquet(destinationbucket)

    print('The file has been succesfully tranformed and moved')

  else:

    s3client.copy_object(CopySource={'Bucket': sourcebucket, 'Key': key}, Bucket=archivebucket, Key=key)
    s3client.delete_object(Bucket=sourcebucket, Key=key)
    print(f'File {key} moved to archive bucket.')

spark.stop()


In [0]:
#there are implications of cost.

# lets make it faster
# increase my cores to 4
# num of executors to 5
# total parallel tasks  = 20

#Start applying optimisation techiques
#broadcast join (for joins)(small df)
#cache (large dataframe)
#repartition > increase the number of parititions (but it shuffles the data)
#coalesce > after tranformation we ll reduce the number of parititions.

#Activate AQE : adaptive query execution : adapt to run time conditions and partition pruning , join selection.
#catalyst optimiser : predicate pushdown (where clause), column pruning (select only the columns that we need), join reordering.



In [0]:
#driver: maintains metadata nd scheduling tasks
#executor: worker proces that run task and holds dat in memory
#cores per executor: number of tasks each executor can run in parallel
#memory per executor: ram avaiable for tasks and caching.
#overhead memory: 20-30%

#executor memory: (jvmheap + spark storage + shuffle + execution)

#75-80% of you memory is going to heap, the rest is overhead.

#rule of thumb:

default memory partition size: 128mb
total partitions needed = datasize / parition size

5gb data
5*1024 = 5120
paritions = 5120 / 128 = 40 paritions

1 parition = 1 task = 1 core

4 cores in 1 executor
for 40 paritions > 10 executors

#for 1gb
number of partitions: 1*1024/128: 8

4 cores per executor : 2 executors.


we need atleast 40 tasks to process the data efficiently
if you have under 5 paritions tasks become heavy.
if you over paritions 1000 for eg. scheduling process will take alot time.


executor memroy sizing (memory + cores)

executor cores: 4 (good balance)
executor memory: 1core = approx : 2-3gb
so 4 cores : 4*2 = 8gb - 12gb per executor


we need 40 partitions each core can run 1 partition at a time.

with 4 cores per executor : each executor can handle 4 partitions in parallel
so 40/4 = 10 executors

driver memory:
driver doesnt do any heavy jobs its primarily coordination

metadata
shuffles
DAG
for 5gb input data we need approximately  4-8 gb (sufficient)



#final config:

data = 5g
paritions =  40 partitions
with 10 executors * 4 cores = 40 tasks at once.
each executor has around 8-12 gg per executor


#adjustments:
if we are doing alot of joins, wide tranformations we need to increase partitions (2-3x): 100-150partitions
if we are doing only filters and narrow transformations then 40-50 is good engough

check spark ui and make changers accordingly.

if there is one paritions that is taking too long to process what do we do ?

1) reparitions
2) salting (Data skewness)

spark-submit

-- num-executors 10\
--executors-cores 4\
--executor-memory 10g\
--driver-memory 6gb

myfile.py






In [0]:
#20 node spark cluster
#each node is of size - 16 cpu cores/ approx 4gb per core/ 64gb ram

#each node has 3 executors : 3*4 : 12 cores per node/ 48gb ram

20nodes * 3 executors : 60 executors

total cpu power: 60*4: 240 cpu cores
total memory: 60executors *4cores*4gb percore: 960 GB

how many parallel tasks can we run

240 cpu cores : 240 parallel tasks

#10gb data from s3 buck, filtering data and how many task to run ?

if we create a df for 10gb file : 80 paritions

we only 20 cores
10 secs to process 128mb data.
first 20 tasks  run in parallel
once these 20 tasks are done then next 20 are executed.

how many cycle does it need to run
it needs 4 cycles.

#is there an possibilty of out of memory issues?

each executor has 4 cores and 16gb

300mb is reserved
40% is used to store variable and data
60 % is uised by spark memory. storage and execution memory : 50:50

30% as execution memory in the memory allocated

6gb memoryis what we have for execution.

6gb/4: 1.25gb we actualy have to process the data.

parition size  = 128mb

so we have sufficient memory to run 128 mb for each parition.







In [0]:
#architecture (Spark) , optimisation techniques, what is reparition, what is coalesce , when do we use them , why do we use them, same with cache, persist with examples
#broadcast join, sortmerge join, when to use them, how does it work. Explain data skewness and what causes this and how can it resolved. what are udf's and how does it work ?
#how to write sql statement in pyspark. how to create manual schema in spark. what was the config for mmeory allocatino for 5gb in spark.

#code: extract data from one source, s3 bucket. do some simple tranformations and move data into another bucket in parquet format.
#second highest salary using window functions in pyspark.



In [0]:
#1) one key(partition) in a join operation has million of records while others have few, causing stage failure or long running tasks. how do we identify and handle this in pyspark?
df.groupbykey('key').count().orderBy(desc('count')) > this will give me the count by key.

1 : 1000000
2 : 15000
3 : 20000

Solution:
repartition
salting


In [0]:
#2) you are reading 100k small files (10kb each) from s3 and the job runs very slow , how to fix this issue.

#merge the files into a single and then partition the data: repartition.
# read it into dynamicframes in glue.



In [0]:
#3) we have a dataset thyat is growing slowly everyday, but only need to process the data when it arrvies.
#what is this and how to solve it.

#incremental Load:
#job bookmarks
#DMS CDC
#timestamping
#versioning
#folder structure


In [0]:
#4 we are joining a large fact table with a small dimension table. what is the best way to join these?

#solution: Broadcast join.



In [0]:
#5)  we are getting alot of data from the client into s3 and in these
#files the client is dropping duplicate files as well by mistake how can we make sure we are not considering the duplicates files.

#having appropriate folder structure.
#verisioning: creates a new version
boto3.list_objects_v2(bucket=sourcebucket)
only pick the latest version files using timestamp
#Activate job bookmarks

#Avoid it at source
#maintain a hashtable deduplication table (mysql/ postgressql)
Create table in mysql
fileid | filename | arrivaldate| hashcode

mydata_01012025.csv

list_objects_v2(bucket=sourcebucket)
for i in reponse:



#track the files that have arrived and move the duplicates into a duplicate folder using s3event and lambda.
#boto3(s3).copyobject(duplicatefolder)





In [0]:
#i have a spark job running and it keeps coming with outofmemory issues. how to solve this problem.

#solution
#reparition
#cache and persist
#allocate more memory
#increase the executor memory


In [0]:
#second highest salary using pyspark code using window functions.




In [0]:
#we have processed the data and we need to put the files into csv. the file size is huge (5GB). this will cause issues as the file size is too big for csv.

#reparition before write and maxrecordsperfile this will write multiple files into s3.



In [0]:
#how will you execute scd type 2 in spark

using withcolumn: startdate, enddate and flag



In [0]:
#what are the different data validation you do in your project and where did you write this code?
#dupe check
#row count validation
#schema validation
#data type validation
#data completeness validation.

#pre tranformation validation

#post tranformation validation: is to check if we have carried all the tranformations as per requirements.

#Athena : adhoc requirements on raw data on glue data catalog we only use when we need like a adhoc check


In [0]:
#we ahve a folder and the customer is dumping both csv and json files in my folder how do i process them together?

#folder structure.
#use unionByName() after confirming the format of the file.

In [0]:
# we have large data 500gb 2 files coming into your s3 bucket.
#how do we optimise this ?

# sortmergejoin
# bucketing pre distribute the data based on join key.

d1.write.bucketby(1000, customerid).sortby(custgomerid).saveasTable('factsales')
